# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
H. Linz  ->  H. Linz  |  ['H. Linz']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
M. Wells  ->  M. Wells  |  ['M. Wells']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
T. Henning  ->  T. Henning  |  ['T. Henning']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 104 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.07162


extracting tarball to tmp_2601.07162...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.07173
extracting tarball to tmp_2601.07173...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.07361


...

Retrieving document from  https://arxiv.org/e-print/2601.07785


extracting tarball to tmp_2601.07785...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2601.07785/ia0_prelaunch.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'sections/conclusions' from 'tmp_2601.07785/sections/conclusions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'sections/euclid_predictions' from 'tmp_2601.07785/sections/euclid_predictions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedt

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure plots/fs2_ia
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure plots/fs2_ia as tmp_2601.07785/./plots/fs2_ia.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07407-b31b1b.svg)](https://arxiv.org/abs/2601.07407) | **The converging gas flow around the infrared dark cloud G28.3**  |
|| <mark>H. Beuther</mark>, et al. -- incl., <mark>C. Gieser</mark>, <mark>H. Linz</mark>, <mark>D. Semenov</mark>, <mark>M. Wells</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *16 pages, 11 figures, accepted for A&A*|
|**Abstract**|            Aims: The G28.37+0.07 star-forming region is a prototypical infrared dark cloud (IRDC) located at the interface of a converging gas flow. This study characterizes the properties of this dynamic gas flow. Methods: Combining data from the Northern Extended Millimeter Array (NOEMA) with single-dish data from the IRAM30m observatory, we mapped large spatial scales (~81pc^2) at high angular resolution (7.0''x2.6'' corresponding ~2.3x10^4au or ~0.1pc) down to core scales. The spectral setup in the 3mm band covers many spectral lines as well as the continuum emission. Results: The data reveal the proposed west-east converging gas flow in all observed dense gas tracers. We estimate a mass-flow rate along that flow around 10^-3M_sun/yr. Comparing these west-east flow rates to infall rates toward sources along the line of sight, the gas flow rates are roughly a factor of 25 greater than than those along the line of sight. This confirms the dominance of longitudinal motions along the converging gas flow in G28.37. For comparison, in the main north-south IRDC formed by the west-east converging gas flow, infall rates along the line of sight are about an order of magnitude greater than those along the west-east flow. In addition to the kinematic analysis, a comparison of CH_3CN-derived gas temperatures with Herschel-derived dust temperatures typically show higher gas temperatures toward high-density sources. We discuss whether mechanical heating from the conversion of the flow's kinetic energy into thermal energy may explain some of the observed temperature differences. Conclusions: The differences between flow rates along the converging flow, perpendicular to it, and toward the sources at the IRDC center indicate that at the interfaces of converging gas flows - where most of the active star formation takes place - originally more directed gas flows can convert into multidirectional infall motions.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07414-b31b1b.svg)](https://arxiv.org/abs/2601.07414) | **Characterization of two new transiting sub-Neptunes and a terrestrial planet around M-dwarf hosts**  |
|| E. Poultourtzidis, et al. -- incl., <mark>I. J. M. Crossfield</mark>, <mark>T. Henning</mark>, <mark>E. Matthews</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *21 pages, 15 figures. Accepted for publication in A&A*|
|**Abstract**|            We report the confirmation of three transiting exoplanets orbiting TOI-1243 (LSPM~J0902+7138), TOI-4529 (G~2--21), and TOI-5388 (Wolf~346) that were initially detected by TESS through ground-based photometry and radial velocity follow-up measurements with CARMENES. The planets present short orbital periods of $4.65$, $5.88$, and $2.59$ days, and they orbit early-M dwarfs (M2.0V, M1.5V, and M3.0V, respectively). We were able to precisely determine the radius of all three planets with a precision of $< 7\, \%$, the mass of TOI-1243 b with a precision of $19\, \%$, and upper mass limits for TOI-4529 b and TOI-5388 b. The radius of TOI-1243 b is $2.33\pm0.12\, {R_{\oplus}}$, its mass is $7.7 \pm 1.5\,{M_{\oplus}}$, and the mean density is $0.61 \pm 0.15 \, {\rho_\oplus}$. The radius of TOI-4529 b is $1.77 ^{+0.09}_{-0.08} \, {R_{\oplus}}$, the $3 {\sigma}$ upper mass limit is $4.9 \, {M_{\oplus}}$, and the $3 {\sigma}$ upper density limit is $0.88\, {\rho_\oplus}$. The third planet, TOI-5388 b, is Earth-sized with a radius of $0.99 ^{+0.07}_{-0.06} \, {R_{\oplus}}$, a $3 {\sigma}$ upper mass limit of $2.2 \, {M_{\oplus}}$, and a $3 {\sigma}$ upper density limit of $2.2\, {\rho_\oplus}$. While TOI-5388 b is most probably rocky, given its Earth-like radius, TOI-1243 b and TOI-4529 b are located in a highly degenerate region in the mass-radius space. TOI-4529 b appears to lean toward a water-world composition. TOI-1243 b has enough mass to host a significant H-He envelope, although a water-world and pure rocky compositions are also consistent with the data. Our analysis indicates that future atmospheric observations using JWST can aid in determining their real composition. The sample of small planets around M dwarfs is widely used to understand planet formation and composition theories, and our study adds three planets to this sample.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07784-b31b1b.svg)](https://arxiv.org/abs/2601.07784) | **Euclid preparation. Testing analytic models of galaxy intrinsic alignments in the Euclid Flagship simulation**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| **|
|**Abstract**|            We model intrinsic alignments (IA) in Euclid's Flagship simulation to investigate its impact on Euclid's weak lensing signal. Our IA implementation in the Flagship simulation takes into account photometric properties of galaxies as well as their dark matter host halos. We compare simulations against theory predictions, determining the parameters of two of the most widely used IA models: the Non Linear Alignment (NLA) and the Tidal Alignment and Tidal Torquing (TATT) models. We measure the amplitude of the simulated IA signal as a function of galaxy magnitude and colour in the redshift range $0.1<z<2.1$. We find that both NLA and TATT can accurately describe the IA signal in the simulation down to scales of $6$-$7 \,h^{-1}\,$Mpc. We measure alignment amplitudes for red galaxies comparable to those of the observations, with samples not used in the calibration procedure. For blue galaxies, our constraints are consistent with zero alignments in our first redshift bin $0.1 < z < 0.3$, but we detect a non-negligible signal at higher redshift, which is, however, consistent with the upper limits set by observational constraints. Additionally, several hydrodynamical simulations predict alignment for spiral galaxies, in agreement with our findings. Finally, the evolution of alignment with redshift is realistic and comparable to that determined in the observations. However, we find that the commonly adopted redshift power-law for IA fails to reproduce the simulation alignments above $z=1.1$. A significantly better agreement is obtained when a luminosity dependence is included, capturing the intrinsic luminosity evolution with redshift in magnitude-limited surveys. We conclude that the Flagship IA simulation is a useful tool for translating current IA constraints into predictions for IA contamination of Euclid-like samples.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07785-b31b1b.svg)](https://arxiv.org/abs/2601.07785) | **Euclid preparation. Calibrated intrinsic galaxy alignments in the Euclid Flagship simulation**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| **|
|**Abstract**|            Intrinsic alignments of galaxies are potentially a major contaminant of cosmological analyses of weak gravitational lensing. We construct a semi-analytic model of galaxy ellipticities and alignments in the \Euclid Flagship simulation to predict this contamination in Euclid's weak lensing observations. Galaxy shapes and orientations are determined by the corresponding properties of the host haloes in the underlying $N$-body simulation, as well as the relative positions of galaxies within their halo. Alignment strengths are moderated via stochastic misalignments, separately for central and satellite galaxies and conditional on the galaxy's redshift, luminosity, and rest-frame colour. The resulting model is calibrated against galaxy ellipticity statistics from the COSMOS Survey, selected alignment measurements based on Sloan Digital Sky Survey samples, and galaxy orientations extracted from the Horizon-AGN hydrodynamic simulation at redshift $z=1$. The best-fit model has a total of 12 alignment parameters and generally reproduces the calibration data sets well within the $1\sigma$ statistical uncertainties of the observations and the \flagship simulation, with notable exceptions for the most luminous sub-samples on small physical scales. The statistical power of the calibration data and the volume of the single \flagship realisation are still too small to provide informative prior ranges for intrinsic alignment amplitudes in relevant galaxy samples. As a first application, we predict that \Euclid end-of-mission tomographic weak gravitational lensing two-point statistics are modified by up to order $10\,\%$ due to intrinsic alignments.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07162-b31b1b.svg)](https://arxiv.org/abs/2601.07162) | **First Submillimeter Lights from Dome A: Tracing the Carbon Cycle in the Feedback of Massive Stars**  |
|| Y. Gong, et al. -- incl., <mark>J. Li</mark>, <mark>X. Zhang</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *This is the accepted manuscript of a paper published in Science Advances. The final version of record is available at DOI: https://doi.org/10.1126/sciadv.aea9433*|
|**Abstract**|            The cycling of carbon between its ionized, atomic, and molecular phases shapes the chemical compositions and physical conditions of the interstellar medium (ISM). However, ground-based studies of the full carbon cycle have been limited by atmospheric absorption. Dome~A, the most promising site for submillimeter astronomy, has long resisted successful submillimeter astronomical observations. Using the 60~cm Antarctic Terahertz Explorer, we present the first successful CO ($4-3$) and [CI] ($^3P_1 - ^3P_0$) mapping observations of two archetypal triggered massive star-formation regions at Dome~A. These data, together with archival [CII], provide the first complete characterization of all three carbon phases in these environments. We find elevated C$^{0}$/CO abundance ratios in high-extinction regions, plausibly driven by deep penetration of intense radiation fields from massive stars into a clumpy ISM. These findings mark a major milestone for submillimeter astronomy at Dome~A and offer valuable insights into the impact of massive star feedback on the surrounding ISM.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07173-b31b1b.svg)](https://arxiv.org/abs/2601.07173) | **Does the radio-active phase of XTE~J1810$-$197 recur following the same evolutionary pattern?**  |
|| Z. Huang, et al. -- incl., <mark>Y. Wu</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *14 pages, 5 figures, Accepted by SCIENCE CHINA Physics, Mechanics & Astronomy*|
|**Abstract**|            Magnetars are the most strongly magnetized compact objects known in the Universe and are regarded as one of the primary engines powering a variety of enigmatic, high-energy transients. However, our understanding of magnetars remains highly limited, constrained by observational sample size and radiative variability. XTE~J1810$-$197, which re-entered a radio-active phase in 2018, is one of only six known radio-pulsating magnetars. Leveraging the distinctive capability for simultaneous dual-frequency observations, we utilized the Shanghai Tianma Radio Telescope (TMRT) to monitor this magnetar continuously at both 2.25 and 8.60~GHz, capturing its entire evolution from radio activation to quenching. This enabled precise characterization of the evolution in its integrated profile, spin frequency, flux density, and spectral index ($\alpha$, defined by $S \propto f^{\alpha}$). The first time derivative of its spin frequency $\dot{\nu}$ passed through four distinct phases -- rapid decrease, violent oscillation, steady decline, and stable recovery -- before returning to its pre-outburst value concomitant with the cessation of radio emission. Remarkably, both the amplitudes and the characteristic time-scales of these $\dot{\nu}$ variations match those observed during the previous outburst that began in 2003, providing the first demonstration that post-outburst rotational evolution and radiative behavior in a magnetar are repeatable. A twisted-magnetosphere model can qualitatively account for this repeatability as well as for the progressive narrowing and abrupt disappearance of the radio pulse radiation, thereby receiving strong observational support.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07361-b31b1b.svg)](https://arxiv.org/abs/2601.07361) | **Strong Evidence for Dark Sector Interactions**  |
|| T.-N. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *18 pages, 8 figures*|
|**Abstract**|            Recent DESI baryon acoustic oscillation data reveal deviations from $\Lambda$CDM cosmology, conventionally attributed to dynamical dark energy (DE). We demonstrate that these deviations are equally, if not better, explained by interactions between dark matter and dark energy (IDE), without requiring a time-varying DE equation of state. Using a unified framework, we analyze two IDE models--coupled quintessence and coupled fluid--against the latest CMB (Planck, ACT, SPT), DESI BAO, and SN (including DES-Dovekie recalibrated) data. Both IDE scenarios show robust evidence for non-vanishing interactions at the 3-5$\sigma$ level, with marginalized constraints significantly deviating from the $\Lambda$CDM limit. This preference persists even under DES-Dovekie SN recalibration, which weakens dynamical DE evidence. Crucially, for the same number of free parameters, IDE models provide fits to low- and high-redshift data that match or exceed the performance of the CPL dynamical DE parametrization. Our results establish IDE as a physically motivated alternative to dynamical DE, highlighting the necessity of future cosmological perturbation measurements (e.g., weak lensing, galaxy clustering) to distinguish between these paradigms.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07623-b31b1b.svg)](https://arxiv.org/abs/2601.07623) | **Impact of Nuclear Reaction Rates on Calcium Production in Population III Stars: A Global Analysis**  |
|| Q. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *22 pages, 22 figures, 4 tables*|
|**Abstract**|            We investigate the sensitivity of calcium production to nuclear reaction rates of a 40 solar-mass Population III star using 1D multi-zone stellar models. A comprehensive nuclear reaction network was constructed, and all $(p,\gamma)$ and $(p,\alpha)$ reaction rates were individually varied by a factor of 10 up and down, identifying 13 preliminary key reactions for calcium production. To propagate the reaction rate uncertainties on calcium production, two sets of Monte Carlo simulations were performed for these key reactions: one adopting STARLIB reaction rates and the other incorporating updated rates from recent experimental data and evaluations. Our results show that Monte Carlo simulations using the updated rates show good agreement with the observed calcium abundance of the extremely iron-poor star SMSS J031300.36-670839.3 within the 68% confidence interval predicted by the models. In contrast, the observed calcium abundance lies marginally outside the 68% C.I. when using the STARLIB rates. Spearman rank-order correlation analysis and SHAP values show that the $(p,\gamma)$ and $(p,\alpha)$ reactions of F18 and F19 exhibit strong coupled effects on calcium production. These reaction-rate uncertainties need to be reduced to constrain the stellar model predictions. Our study provides insights for future nuclear physics experiments aimed at reducing reaction rate uncertainties in the nucleosynthesis of Population III Stars. Additionally, comparisons between 20 solar-mass and 40 solar-mass Population III stellar models confirm that the latter, with updated reaction rates, is more capable of reproducing the observed Ca abundance and [Ca/Mg] ratio.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07688-b31b1b.svg)](https://arxiv.org/abs/2601.07688) | **Brightest GRB flare observed in GRB 221009A: bridge the last gap between flare and prompt emission in GRB**  |
|| Z.-H. Yu, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-01-13*|
|*Comments*| *27 pages, 5 figures, accepted for publication in ApJL*|
|**Abstract**|            Flares are usually observed during the afterglow phase of Gamma-Ray Bursts (GRBs) in soft X-ray, optical and radio bands, but rarely in gamma-ray band. Despite the extraordinary brightness, GECAM-C has accurately measured both the bright prompt emission and flare emission of GRB 221009A without instrumental effects, offering a good opportunity to study the relation between them. In this work, we present a comprehensive analysis of flare emission of GRB 221009A, which is composed of a series of flares. Among them, we identify an exceptionally bright flare with a record-breaking isotropic energy $E_{\rm iso} = 1.82 \times 10^{53}$ erg of GRB flares. It exhibits the highest peak energy ever detected in GRB flares, $E_{\rm peak} \sim 300$ keV, making it a genuine gamma-ray flare. It also shows rapid rise and decay timescales, significantly shorter than those of typical X-ray flares observed in soft X-ray or optical band, but comparable to those observed in prompt emissions. Despite these exceptional properties, the flare shares several common properties with typical GRB flares. We note that this is the first observation of a GRB flare in the keV-MeV band with sufficiently high temporal resolution and high statistics, which bridges the last gap between prompt emission and flare.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.07407/./plot_noema_followup.png', 'tmp_2601.07407/./pv_h13co+.png', 'tmp_2601.07407/./pv_hco+.png', 'tmp_2601.07407/./pv_sio.png', 'tmp_2601.07407/./T_ch3cn_v2.png', 'tmp_2601.07407/./mom1.png', 'tmp_2601.07407/./mom1_merged.png']
copying  tmp_2601.07407/./plot_noema_followup.png to _build/html/
copying  tmp_2601.07407/./pv_h13co+.png to _build/html/
copying  tmp_2601.07407/./pv_hco+.png to _build/html/
copying  tmp_2601.07407/./pv_sio.png to _build/html/
copying  tmp_2601.07407/./T_ch3cn_v2.png to _build/html/
copying  tmp_2601.07407/./mom1.png to _build/html/
copying  tmp_2601.07407/./mom1_merged.png to _build/html/
exported in  _build/html/2601.07407.md
    + _build/html/tmp_2601.07407/./plot_noema_followup.png
    + _build/html/tmp_2601.07407/./pv_h13co+.png
    + _build/html/tmp_2601.07407/./pv_hco+.png
    + _build/html/tmp_2601.07407/./pv_sio.png
    + _build/html/tmp_2601.07407/./T_ch3cn_v2.png
    + _build/html/tmp_2601.07407/./mom1.png
    + _build/ht

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pdeg}{\ifmmode \setbox0=\hbox{^{\circ}}\rlap{\hskip.11\wd0 .}^{\circ}$
$  \else \setbox0=\hbox{^{\circ}}\rlap{\hskip.11\wd0 .}^{\circ}\fi}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# The converging gas flow around the infrared dark cloud G28.37

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.07407-b31b1b.svg)](https://arxiv.org/abs/2601.07407)<mark>Appeared on: 2026-01-13</mark> -  _16 pages, 11 figures, accepted for A&A_

</div>
<div id="authors">

<mark>H. Beuther</mark>, et al. -- incl., <mark>C. Gieser</mark>, <mark>H. Linz</mark>, <mark>D. Semenov</mark>, <mark>M. Wells</mark>

</div>
<div id="abstract">

**Abstract:** How dense clouds and star-forming regions form out of the      dynamical interstellar medium is at the heart of star formation      research. The G28.37+0.07 star-forming region is a prototypical infrared dark cloud      (IRDC) located at the interface of a      converging gas flow. This study characterizes the properties of      this dynamic gas flow. Combining data from the Northern Extended Millimeter Array      (NOEMA) with single-dish data from the IRAM 30 m observatory, we mapped large spatial scales ( $\sim$ 81 pc $^2$ ) at high      angular resolution ( $7.0"\times 2.6"$ corresponding $\sim$ 2.3 $\times 10^4$ au or $\sim$ 0.1 pc) down to core      scales. The spectral setup in the 3 mm band covers many      spectral lines as well as the continuum emission. The data clearly reveal the proposed west-east converging gas flow in  all observed dense gas tracers. We estimate a mass-flow rate along  that flow around $10^{-3}$ M $_{\odot}$ yr $^{-1}$ . Comparing  these west-east flow rates to infall rates toward sources along the  line of sight, the gas flow rates are roughly a factor  of 25 greater than than those along the line of sight (roughly  perpendicular to the west-east flow). This confirms the dominance of  longitudinal motions along the converging gas flow in this  region. For comparison, in the main north-south IRDC formed  by the west-east converging gas flow, infall rates along the  line of sight are about an order of magnitude greater than those along the  west-east flow. In addition to the kinematic analysis, a comparison of  $CH_3$ CN-derived gas temperatures with Herschel-derived dust  temperatures typically show higher gas temperatures toward  high-density sources. We discuss whether mechanical heating from the  conversion of the flow's kinetic energy into thermal energy may  explain some of the observed temperature differences. Our analysis of the G28.37+0.07 converging gas flow shows that such  structures can indeed form and feed typical high-mass  star-forming regions in the Milky Way. The differences between flow  rates along the converging flow, perpendicular to it, and toward  the sources at the IRDC center indicate that at the  interfaces of converging gas flows -- where most of the active star  formation takes place -- originally more directed gas flows can  convert into multidirectional infall motions.

</div>

<div id="div_fig1">

<img src="tmp_2601.07407/./plot_noema_followup.png" alt="Fig9.1" width="25%"/><img src="tmp_2601.07407/./pv_h13co+.png" alt="Fig9.2" width="25%"/><img src="tmp_2601.07407/./pv_hco+.png" alt="Fig9.3" width="25%"/><img src="tmp_2601.07407/./pv_sio.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Velocity comparison between large-scale APEX [CI] and
  small-scale NOEMA emission data. Top two panels: Reproduction of the [CI] first-moment map and pv cut
  from \citet{beuther2020}. Top-left: Color scale and contours
  showing the [CI] first-moment map and ATLASGAL 870 $\mu$m emission
  (contour levels starting at $4\sigma$ and continuing in $8\sigma$ steps,
  $1\sigma$=50 mJy beam$^{-1}$). A scale bar is shown as well.
    Top-right: Position-velocity diagram along the white arrow
  in the top-left panel from high to low latitudes (high latitude
  offset 0). The full white lines mark the center position of the single-dish filament, and the dotted lines indicate the
  approximate extent of the corresponding pv cuts from the NOEMA
  data. Bottom row: Three pv diagrams in color, with contours along the west-east line (offset 0 indicates west) shown in
  Fig. \ref{mom1} in H$^{13}$CO$^+$, HCO$^+$, and SiO. The color scale
  shows the corresponding NOEMA+30 m data, and the contours show
  the 30 m data only. The red contours in the middle HCO$^+$ panel
  also show the corresponding H$^{13}$CO$^+$ emission (30 m-only
  data). Contour levels are in $4\sigma$ steps. (*pv*)

</div>
<div id="div_fig2">

<img src="tmp_2601.07407/./T_ch3cn_v2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Temperature maps. Left: Temperature map from combined fitting of $CH_3$CN$(5_k-4_k)$ and
  $(4_k-3_k)$$k$-ladders ($k$ from 0 to 4). Middle: Dust temperature map from Herschel far-infrared
  data \citep{marsh2017}. Right: $CH_3$CN temperature map, smoothed to
  the same $12"$ spatial resolution of the Herschel map. The
  contours outline the NOEMA 3.6 mm continuum emission; contour
  levels are from 0.4 to 1.6 mJy beam$^{-1}$($1\sigma \sim
  0.1$ mJy beam$^{-1}$). The source numbers and a scale bar are shown
  in the left panel. (*temp*)

</div>
<div id="div_fig3">

<img src="tmp_2601.07407/./mom1.png" alt="Fig8.1" width="50%"/><img src="tmp_2601.07407/./mom1_merged.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** First-moment maps (intensity-weighted peak velocities) for
  G28. Top: 30 m observations. Bottom: Merged NOEMA+30m data (except NH
2
D, which shows NOEMA-only data as that line was
  not covered by the 30 m observations). All maps were created by clipping the
  data below the $3\sigma$ level. The contours on the 30 m data show
  870 $\mu$m continuum \citep{schuller2009} in $3\sigma$ steps of
  0.15 Jy beam$^{-1}$. The contours on the NOEMA+30 m data show the
  NOEMA-only 3.6 mm continuum from 0.4 to to
  1.6 mJy beam$^{-1}$($1\sigma\sim$0.1 mJy beam$^{-1}$). Molecules are labeled in all
  panels, and the beam and scale bar are shown in the top-left
  panels. The H$^{13}$CO$^+$ panels outline the west-east pv cut
  presented in Fig. \ref{pv}. (*mom1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.07407"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\plaa}{TOI\mbox{-}1243 b}$
$\newcommand{\plab}{TOI\mbox{-}4529 b}$
$\newcommand{\plac}{TOI\mbox{-}5388 b}$
$\newcommand{\splaa}{TOI\mbox{-}1243}$
$\newcommand{\splab}{TOI\mbox{-}4529}$
$\newcommand{\splac}{TOI\mbox{-}5388}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\pdcsap}{\textit{PDCSAP}}$
$\newcommand{\muscat}{\textit{MuSCAT2}}$
$\newcommand{\kms}{\mbox{km s^{-1}}}$
$\newcommand{\arraystretch}{1.35}$</div>



<div id="title">

# Characterization of two new transiting sub-Neptunes and a terrestrial planet around M-dwarf hosts

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.07414-b31b1b.svg)](https://arxiv.org/abs/2601.07414)<mark>Appeared on: 2026-01-13</mark> -  _21 pages, 15 figures. Accepted for publication in A&A_

</div>
<div id="authors">

E. Poultourtzidis, et al. -- incl., <mark>I. J. M. Crossfield</mark>, <mark>T. Henning</mark>, <mark>E. Matthews</mark>

</div>
<div id="abstract">

**Abstract:** We report the confirmation of three transiting exoplanets orbiting $\splaa$ (LSPM J0902+7138), $\splab$ (G 2--21), and $\splac$ (Wolf 346) that were initially detected by $\tess$ through ground-based photometry and radial velocity follow-up measurements with CARMENES. The planets present short orbital periods of $4.65$ , $5.88$ , and $2.59$ days, and they orbit early-M dwarfs (M2.0 V, M1.5 V, and M3.0 V, respectively). We were able to precisely determine the radius of all three planets with a precision of $< 7  \%$ , the mass of $\plaa$ with a precision of $19  \%$ , and upper mass limits for $\plab$ and $\plac$ . The radius of $\plaa$ is $2.33\pm0.12$ $\mathrm{R_{\oplus}}$ , its mass is $7.7 \pm 1.5 \mathrm{M_{\oplus}}$ , and the mean density is $0.61 \pm 0.15$ $\mathrm{\rho_\oplus}$ . The radius of $\plab$ is $1.77 ^{+0.09}_{-0.08}   \mathrm{R_{\oplus}}$ , the $3 \mathrm{\sigma}$ upper mass limit is $4.9   \mathrm{M_{\oplus}}$ , and the $3 \mathrm{\sigma}$ upper density limit is $0.88  \mathrm{\rho_\oplus}$ . The third planet, $\plac$ , is Earth-sized with a radius of $0.99 ^{+0.07}_{-0.06}   \mathrm{R_{\oplus}}$ , a $3 \mathrm{\sigma}$ upper mass limit of $2.2   \mathrm{M_{\oplus}}$ ,and a $3 \mathrm{\sigma}$ upper density limit of $2.2  \mathrm{\rho_\oplus}$ . While $\plac$ is most probably rocky, given its Earth-like radius, $\plaa$ and $\plab$ are located in a highly degenerate region in the mass-radius space. $\plab$ appears to lean toward a water-world composition. $\plaa$ has enough mass to host a significant H-He envelope, although a water-world and pure rocky compositions are also consistent with the data. Our analysis indicates that future atmospheric observations using JWST can aid in determining their real composition. The sample of small planets around M dwarfs is widely used to understand planet formation and composition theories, and our study adds three planets to this sample.

</div>

<div id="div_fig1">

<img src="tmp_2601.07414/./diagrams/periodograms/all_TOI-1243.png" alt="Fig5" width="100%"/>

**Figure 5. -** Periodograms of the spectroscopic data of $\plaa$. The first and second panels show the RV time series, and the residuals of the RV after the best-fit model. The additional panels show the periodograms of activity indicators. The yellow vertical line represents the expected 4.66 d transit period of the planet from the photometric analysis. (*fig:per1243*)

</div>
<div id="div_fig2">

<img src="tmp_2601.07414/./diagrams/mr_plots/m2_planets_log.png" alt="Fig4" width="100%"/>

**Figure 4. -** Mass-radius diagram for the three discussed planets in comparison with validated exoplanets around M dwarfs with masses of $M_{\rm p} < 30$ M$_{\oplus}$ and $R_{\rm p} < 4$ R$_{\oplus}$. The exoplanet data were downloaded from NASA Exoplanet Archive on 2025 August 2. The black dots highlight planets characterized with CARMENES. The diagram features several compositional lines as computed by [Zeng, et. al (2019)](https://ui.adsabs.harvard.edu/abs/2019PNAS..116.9723Z). The $50\%$ Earth-like + $50\%$ $H_2$O and $49.95\%$ Earth-like + $49.95\%$ $H_2$O + 0.1 $H_2$ compositional lines are plotted for an equilibrium temperature of 500 K, that is, the closest to the derived temperature of the exoplanets under analysis. (*fig:mr*)

</div>
<div id="div_fig3">

<img src="tmp_2601.07414/./diagrams/periodograms/all_TOI-4529.png" alt="Fig6" width="100%"/>

**Figure 6. -** Same as Fig.  \ref{fig:per1243}, but for $\plab$. The red marked power represent periods with FAP<$0.13\%$.The yellow vertical line represents the expected 5.38 d transit period of the planet from the photometric analysis. (*fig:per4529*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.07414"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\DESKIDS}{DES\&KiDs\xspace}$
$\newcommand{\Kai}{H26\xspace}$
$\newcommand{\kai}[1]{\textcolor{purple}{\bf Kai: #1}}$
$\newcommand{\rp}[1]{\textcolor{pink}{\bf rp: #1}}$
$\newcommand{\sandrine}[1]{\textcolor{darkraspberry}{[SC:\bf{#1}]}}$
$\newcommand{\ci}{\mathrm{i}}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\orcid}[1]$
$\newcommand{\der}{\mathrm{d}}$</div>



<div id="title">

# $\Euclid$ preparation. Testing analytic models of galaxy intrinsic alignments in the $\Euclid$ Flagship simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.07784-b31b1b.svg)](https://arxiv.org/abs/2601.07784)<mark>Appeared on: 2026-01-13</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We model intrinsic alignments (IA) in $\Euclid$ 's Flagship simulation to investigate its impact on $\Euclid$ 's weak lensing signal. Our IA implementation in the Flagship simulation takes into account photometric properties of galaxies as well as their dark matter host halos. The simulation parameters are calibrated using combined constraints from observations and cosmological hydrodynamical simulations. We compare simulations against theory predictions, determining the parameters of two of the most widely used IA models: the Non Linear Alignment (NLA) and the Tidal Alignment and Tidal Torquing (TATT) models.We measure the amplitude of the simulated IA signal as a function of galaxy magnitude and colour in the redshift range $0.1<z<2.1$ , similar to $\Euclid$ 's main galaxy sample. We find that both NLA and TATT can accurately describe the IA signal in the simulation down to scales of $6$ -- $7  h^{-1} $ Mpc. We measure alignment amplitudes for red galaxies comparable to those of the observations, with samples not used in the calibration procedure. For blue galaxies, our constraints are consistent with zero alignments in our first redshift bin $0.1 < z < 0.3$ , but we detect a non-negligible signal at higher redshift, which is, however, consistent with the upper limits set by observational constraints. Additionally, several hydrodynamical simulations predict alignment for spiral galaxies, in agreement with our findings. Finally, the evolution of alignment with redshift is realistic and comparable to that determined in the observations. However, we find that the commonly adopted redshift power-law for IA fails to reproduce the simulation alignments above $z=1.1$ . A significantly better agreement is obtained when a luminosity dependence is included, capturing the intrinsic luminosity evolution with redshift in magnitude-limited surveys.We conclude that the Flagship IA simulation is a useful tool for translating current IA constraints into predictions for IA contamination of $\Euclid$ -like samples.

</div>

<div id="div_fig1">

<img src="tmp_2601.07784/./figures/Lr_vs_A1.png" alt="Fig16" width="100%"/>

**Figure 16. -** IA linear amplitude versus luminosity for the NLA model. The low-$z$ sample corresponds to the redshift slices below $z = 0.5$, while the high-$z$ sample corresponds to the redshift slices between $0.5 < z < 0.9$. The luminosity is computed as $\logten( L / L_0) = -(M_\text{r} - M_0)/2.5$. The shaded areas correspond to the 68$\%$ confidence regions, determined by propagating the errors on the power-law amplitudes and indices. The observational measurements are given in: SDSS-Main-red, SDSS-LRG and MegaZ LRG $\ci$tep{Joachimi11}, Gama-red and SDSS-red $\ci$tep{Johnston19}, LOWZ $\ci$tep{Singh15}, KiDS-1000 LRG dense and lum $\ci$tep{fortuna21b}, DES Y3 RML and RMH $\ci$tep{Samuroff22}, CMASS and eBOSS LRG $\ci$tep{FABIBI2024}. (*fig:LR_vs_A1_obs*)

</div>
<div id="div_fig2">

<img src="tmp_2601.07784/./figures/Z_vs_A1_flagship.png" alt="Fig17" width="100%"/>

**Figure 17. -** Redshift evolution of the linear IA amplitude. Each point represents the fit performed with the NLA-1 model to either the density samples (black), red samples (red), or blue samples (blue). Here, each sample is subsampled (if possible) to have the same number density, $n = 4 \times 10^{-3} h^{-3}$Mpc$^{3}$. *Left*: the solid lines represent the best-fit power law of Eq. \eqref{eq:znla}, with constraints given in Table \ref{tab:znla}. *Right*: the solid lines now represent the best fit of Eq. \eqref{eq:enla}. (*fig:IA_vs_red*)

</div>
<div id="div_fig3">

<img src="tmp_2601.07784/./figures/color_fs2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Colour-magnitude diagram of Flagship galaxies at redshift $z=0.2$ and $z=0.8$. We present the 68, 95, and 99$\%$ percentiles of the distributions. The dashed vertical line represents the cut that we used to define red and blue galaxy samples, i.e. $u-r = 1.32$. We shifted the rest-frame magnitude by $-4$ at redshift $z=0.8$ for clarity. (*fig:color_distribution*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.07784"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\eq}[1]{\begin{equation}  #1 \end{equation}}$
$\newcommand{\eqa}[1]{\begin{eqnarray}   #1 \end{eqnarray}}$
$\newcommand{\sigMf}{\sigma_{\rm MF}\xspace}$
$\newcommand{\sigMfC}{\sigma_{\rm MF}^{\rm cent}\xspace}$
$\newcommand{\sigMfS}{\sigma_{\rm MF}^{\rm sat}\xspace}$
$\newcommand{\wgp}{w_{\rm g+}\xspace}$
$\newcommand{\wgg}{w_{\rm gg}\xspace}$
$\newcommand{\etaAC}{\eta_{\rm A/C}\xspace}$
$\newcommand{\etaA}{\eta_{\rm A}\xspace}$
$\newcommand{\etaC}{\eta_{\rm C}\xspace}$
$\newcommand{\mpch}{h^{-1}\mathrm{Mpc}\xspace}$
$\newcommand{\chisqtot}{{\chi}^2_{\rm tot}\xspace}$
$\newcommand{\cost}{C\xspace}$
$\newcommand{\flagship}{Flagship\xspace}$
$\newcommand{\ci}{\mathrm{i}}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation: Calibrated intrinsic galaxy alignments in the Euclid Flagship simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.07785-b31b1b.svg)](https://arxiv.org/abs/2601.07785)<mark>Appeared on: 2026-01-13</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Intrinsic alignments of galaxies are potentially a major contaminant of cosmological analyses of weak gravitational lensing.We construct a semi-analytic model of galaxy ellipticities and alignments in the $\Euclid$ Flagship simulation to predictthis contamination in Euclid's weak lensing observations. Galaxy shapes and orientations are determined by the corresponding properties of the host haloes in the underlying $N$ -body simulation, as well as the relative positions of galaxies within their halo. Alignment strengths are moderated via stochastic misalignments, separately for central and satellite galaxies and conditional on the galaxy's redshift, luminosity, and rest-frame colour. The resulting model is calibrated against galaxy ellipticity statistics from the COSMOS Survey, selected alignment measurements based on Sloan Digital Sky Survey samples, and galaxy orientations extracted from the Horizon-AGN hydrodynamic simulation at redshift $z=1$ .The best-fit model has a total of 12 alignment parameters and generally reproduces the calibration data sets well within the $1\sigma$ statistical uncertainties of the observations and the $\flagship$ simulation, with notable exceptions for the most luminous sub-samples on small physical scales. The statistical power of the calibration data and the volume of the single $\flagship$ realisation are still too small to provide informative prior ranges for intrinsic alignment amplitudes in relevant galaxy samples.As a first application, we predict that $\Euclid$ end-of-mission tomographic weak gravitational lensing two-point statistics are modified by up to order $10 \%$ due to intrinsic alignments.

</div>

<div id="div_fig1">

<img src="tmp_2601.07785/./plots/wgp_fs2_vs_obs_fixed_sigma.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
        Projected galaxy-ellipticity cross-correlation between a density sample and various shape samples. The four left panels show results based on shapes from the luminosity sub-samples, with the full LOWZ sample as the density sample. The two right panels display results for SDSS, using the Main sample as the density sample and colour sub-samples as shape samples. Black dots represent observational data, while coloured lines represent $\flagship$ simulations for three simplified intrinsic alignment (IA) models, each using different fixed values for $\sigma_{\rm MF}^{\rm cent}$ and $\sigma_{\rm MF}^{\rm sat}$.
     (*fig:wgp_lowz_fixed_sigma*)

</div>
<div id="div_fig2">

<img src="tmp_2601.07785/./plots/zmc_distribution_fs2mocks_Euclid_SDSS_LOWZ_HAGN.png" alt="Fig1" width="100%"/>

**Figure 1. -** Redshift, magnitude, colour distributions of mock samples, used for calibrating the IA
        model parameters for galaxy misalignment. Contours enclose $68\%$ and $95\%$ of the distributions. (*fig:zmc_distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2601.07785/./plots/eta_hagn_fixed_sigma.png" alt="Fig12" width="100%"/>

**Figure 12. -** 3D IA cross-correlations ($\eta$, as defined in Eq. \ref{eq:eta})
   between a main density sample and the orientations of galaxies in three
   luminosity-limited sub-samples. Orientations are given by the 3D major
   and minor axes $A$ and $C$. Black dots show measurements
   from the HAGN simulation at $z=1.0$ with errors indicating the standard deviation.
   coloured lines show the results from mock HAGN samples from $\flagship$ for three
   simplified IA models with different combinations of fixed values for
   $\sigma_{\rm MF}^{\rm cent}$ and $\sigma_{\rm MF}^{\rm sat}$, analogously to
   Fig. \ref{fig:wgp_lowz_fixed_sigma}. Errors for $\flagship$ measurements are
   similar to those displayed in Fig. \ref{fig:eta_hagn_flagship} and are not shown for clarity. (*fig:eta_hagn_flagship_fixed_sigma*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.07785"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

113  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
